In [40]:
import os
import pandas as pd
from datetime import datetime as dt 
from datetime import date
import string
import math as mt
import numpy as np
from pandasql import sqldf

In [41]:
os.chdir('C:\\Users\\prajwal.d1\\Desktop\\Pendency')
RTO_RVP=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\totalShipmentsPendingInTheHub_2025-Apr-21_21.csv")
UT=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\untraceableShipments_2025-Apr-22_05.csv")

C:\Users\prajwal.d1\AppData\Local\Temp\ipykernel_23300\4004783290.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  RTO_RVP=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\totalShipmentsPendingInTheHub_2025-Apr-21_21.csv")


In [42]:
Map = pd.read_csv(r"C:\Users\prajwal.d1\Desktop\Pendency\ODH Model pincode Tracker.csv")
New1 = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Pendency\FMA'25 FA Accounting Data 21.04.25.xlsb",engine="pyxlsb", sheet_name="Raw Data")  # FA Accounting Data
New2 = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Pendency\NFA-21.04.25.xlsb", engine="pyxlsb", sheet_name="RAW") 
New3 = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Pendency\Gunti_Raw2025-04-22.xlsx")  # Overall >3days
New4 = pd.read_csv(r"C:\Users\prajwal.d1\Desktop\Pendency\CV_Code_Final\CV_Data_MST2025-04-22.csv",encoding="ISO-8859-1")  # CV Multitrack Data
New5 = pd.read_csv(r"C:\Users\prajwal.d1\Desktop\Pendency\BRSNR_Mulitracking.csv",encoding="ISO-8859-1")  # Try other encodings if needed


C:\Users\prajwal.d1\AppData\Local\Temp\ipykernel_23300\1887372095.py:6: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  New5 = pd.read_csv(r"C:\Users\prajwal.d1\Desktop\Pendency\BRSNR_Mulitracking.csv",encoding="ISO-8859-1")  # Try other encodings if needed


In [43]:
Map.columns

Index(['ODH/MDH Name', 'Type', 'Vendor', 'Status', 'Zone', 'City', 'State'], dtype='object')

In [44]:
Map=Map[['ODH/MDH Name','Vendor']]
Map.columns=['HubName','Vendor Name']
Map['HubName']=Map['HubName'].apply(lambda x:x.upper())
Map['Vendor Name']=Map['Vendor Name'].apply(lambda x:x.upper())
Map=Map.drop_duplicates()
RTO_RVP['HubName']=RTO_RVP['HubName'].apply(lambda x:x.upper())
RTO_RVP=RTO_RVP.merge(Map,how='left',on=['HubName'])
RTO_RVP=RTO_RVP.dropna(subset=['Vendor Name'])

In [45]:
RTO_RVP['RefTime']=np.where(RTO_RVP['TypeOfShipment']=='RTO',RTO_RVP['RTOCreatedDateTime'],RTO_RVP['LastUpdatDateTime'])
RTO_RVP=RTO_RVP[(RTO_RVP['RefTime']!='---')]
RTO_RVP['TimeDelta']=dt.now()-pd.to_datetime(RTO_RVP['RefTime'])
RTO_RVP['Ageing']=RTO_RVP['TimeDelta'].apply(lambda x: float(x.days)+float(x.seconds)/(60*60*24))

In [46]:
RTO_RVP['Month']=pd.DatetimeIndex(RTO_RVP['RefTime']).month
RTO_RVP['Year']=pd.DatetimeIndex(RTO_RVP['RefTime']).year
RTO_RVP=RTO_RVP.drop(['RefTime','TimeDelta'],axis=1)
RTO_RVP['Month']=RTO_RVP['Month'].astype(str)
RTO_RVP['Length']=RTO_RVP['Month'].apply(lambda x:len(x))
RTO_RVP['Month']=np.where(RTO_RVP['Length']==2,RTO_RVP['Year'].astype(str)+RTO_RVP['Month'].astype(str),
                          RTO_RVP['Year'].astype(str)+"0"+RTO_RVP['Month'].astype(str))
RTO_RVP['Month']=RTO_RVP['Month'].apply(lambda x: int(x))

In [47]:
RTO_RVP_Track=RTO_RVP[(RTO_RVP['Ageing']>3)&(RTO_RVP['Month']>202501)]

In [48]:
UT=UT.drop(['Ageing'],axis=1)
UT['HubName']=UT['HubName'].apply(lambda x:x.upper())
UT=UT.merge(Map,how='left',on=['HubName'])
UT=UT.dropna(subset=['Vendor Name'])
UT['TimeDelta']=dt.now()-pd.to_datetime(UT['CurrentStatusUpdateDateTime'])
UT['Ageing']=UT['TimeDelta'].apply(lambda x: float(x.days)+float(x.seconds)/(60*60*24))

In [49]:
UT['Month']=pd.DatetimeIndex(UT['CurrentStatusUpdateDateTime']).month
UT['Year']=pd.DatetimeIndex(UT['CurrentStatusUpdateDateTime']).year
UT=UT.drop(['TimeDelta'],axis=1)
UT['Month']=UT['Month'].astype(str)
UT['Length']=UT['Month'].apply(lambda x:len(x))
UT['Month']=np.where(UT['Length']==2,UT['Year'].astype(str)+UT['Month'].astype(str),
                     UT['Year'].astype(str)+"0"+UT['Month'].astype(str))
UT['Month']=UT['Month'].apply(lambda x: int(x))
UT_Track=UT[(UT['Ageing']>3)&(UT['Month']>202501)]

In [50]:
New3.columns

Index(['Unnamed: 0', 'ShipmentId', ' Status', ' TotalPrice', 'In Cr',
       ' DeliveryHub', ' CurrentHub', 'Mappped hub', 'Asset', 'Owner',
       'Remarks', ' LatestUpdateTime', ' BagId', ' BagStatus', 'ageing',
       'Month', ' NumberOfAttempts', 'RTO DATE', 'Vertical', 'source', 'Zone',
       'GM', 'RM', 'AM / POC', 'seller_id', 'Eagle Eye Remarks Update-25TH',
       'Issue Type', 'Ticket No', 'Amout Recovery Status (Lost)',
       'Recovery Status', 'Hub Reasn Code (FST)'],
      dtype='object')

In [51]:
Map.columns=['cl_current_hub_name','Vendor Name'] #vlookup current hub & vendor name 
New1 = New1.dropna(subset=['cl_current_hub_name']) #current hub NA/Blanks Removed
New1['cl_current_hub_name']=New1['cl_current_hub_name'].apply(lambda x:x.upper()) #Uppercase Currenthub
#New1=New1.merge(Map,how='left',on=['current_hub'])
New1 = New1.merge(Map, how='left', on=['cl_current_hub_name'], suffixes=('', '_map')) #vlookup clcurrenthub & Vendor Name
New1_Track=New1[(New1['Pendency/BRSNR']=='Pendency') & (New1['Vendor Name'].isna()==False)] # Keep Pendeny @C-Pendency/BRSNR , Remove NA
Map.columns=['current_hub','Vendor Name']

New2 = New2.dropna(subset=['current_hub'])
New2['current_hub']=New2['current_hub'].apply(lambda x:x.upper())
New2=New2.merge(Map,how='left',on=['current_hub'], suffixes=('', '_map'))
New2_Temp=New2[(New2['review']=='DH pendency') | (New2['review']=='MP pendency') | (New2['review']=='DH Pendency') | (New2['review']=='MP Pendency')] #Fliter and keep only 
New2_Track=New2_Temp[(New2_Temp['Vendor Name'].isna()==False)]
Map.columns=['Mapped hub','Vendor Name']

New3['Mappped hub'] = New3['Mappped hub'].str.upper()
New3 = New3.merge(Map, how='left', left_on='Mappped hub', right_on='Mapped hub', suffixes=('', '_map'))
New3_Track = New3[New3['Vendor Name'].notna()]

#New3['Mapped hub']=New3['Mapped hub'].apply(lambda x:x.upper())
#New3 = New3.merge(Map, how='left', on=['Mapped hub'], suffixes=('', '_map1'))
#New3 = New3.merge(Map, how='left', on=['Mapped hub'], suffixes=('', '_map2'))
#New3_Track = New3[New3['Vendor Name_map2'].notna()]
#New3=New3.merge(Map,how='left',on=['Mapped hub'])
#New3 = New3.merge(Map, how='left', on=['Mapped hub'], suffixes=('_left', '_right'))
#New3_Track=New3[(New3['Vendor Name'].isna()==False)]

In [52]:
RTO_RVP_TIDs = pd.DataFrame(RTO_RVP_Track['ShipmentId'])
UT_TIDs = pd.DataFrame(UT_Track['ShipmentId'])
New1_TIDs = pd.DataFrame(New1_Track['vendor_tracking_id'])
New2_TIDs = pd.DataFrame(New2_Track['vendor_tracking_id'])
New3_TIDs = pd.DataFrame(New3_Track['ShipmentId'])
New4_Track = New4 #saturday
New4_TIDs = pd.DataFrame(New4_Track['ShipmentId']) #Adding New4 DataFrame   #saturday
New5_Track = New5
New5_TIDs = pd.DataFrame(New5_Track['ShipmentId']) 


In [53]:
New1_TIDs.columns = ['ShipmentId']
New2_TIDs.columns = ['ShipmentId']
New4_TIDs.columns = ['ShipmentId'] #Saturday
New5_TIDs.columns = ['ShipmentId']

In [54]:
Track = pd.concat([RTO_RVP_TIDs, UT_TIDs, New1_TIDs, New2_TIDs, New3_TIDs, New4_TIDs,New5_TIDs])
# Track = pd.concat([RTO_RVP_TIDs, UT_TIDs, New1_TIDs, New2_TIDs, New3_TIDs]) #Saturday
Track = Track.drop_duplicates()


In [55]:
Track.to_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\All_Multitrack_TIDs.csv")
#Track.to_csv("Final_Tracking_IDs.csv", index=False)  

In [56]:
Automa=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\data_Automa.csv")
Manual=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\AllFinal.csv")

C:\Users\prajwal.d1\AppData\Local\Temp\ipykernel_23300\1192926215.py:2: DtypeWarning: Columns (5,6,17,20,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  Manual=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\AllFinal.csv")


In [57]:
Manual1 = Manual

In [58]:
Gunti_Multitracked = Manual[Manual['ShipmentId'].isin(New3['ShipmentId'])].drop_duplicates(subset=['ShipmentId']) #saturday
Gunti_Multitracked

,ShipmentId,TierType,Type,ReverseShipmentId,ForwardReverseType,TotalPrice,DeliveryPincode,Status,DeliveryHub,CurrentHub,...,NumberOfAttempts,CsNotes,HubNotes,OnHoldByOpsReason,OnHoldByOpsDate,BagId,BagStatus,ConsignmentId,OrderId,ItemDescription
6,FMPP2987193212,ECONOMY,PP,NaN,NORMAL_FORWARD,821,382430,Undelivered_Request_For_Reschedule,VastralMPHub_ADI,PrincyScienceCityODH_SCI,...,1,Marked_As_RTO,Updated by CRON,NaN,NaN,PH-B2-AHDABD-10895248,TERMINATED,229040921,OD334147205627037100,KRB KRB Idli Cooker Multi Kadai Steamer 6 Plat...
11,FMPP2986133055,ECONOMY,PP,NaN,NORMAL_FORWARD,488,395006,Expected,SuratMPHub_STV,SherkhiBachauODH_BAC,...,0,Marked_As_RTO,NaN,NaN,NaN,EBHJRCB-710620,REACHED,229306751,OD434129984670117100,AATIVYA Men Solid Casual Green Shirt.
13,FMPP2982774284,ECONOMY,PP,NaN,NORMAL_FORWARD,285,560102,Undelivered_Not_Attended,EcityMPHub_BLR,NarayanpurHub_NRP,...,0,Marked_As_RTO,NaN,NaN,NaN,ERAICHG-3539357,CLOSED,NaN,OD334128672930329100,Village Organica Sattu Powder | Dry Roasted | ...
14,FMPP2984406516,ECONOMY,PP,NaN,NORMAL_FORWARD,1258,562122,Undelivered_Request_For_Reschedule,Malur BTS Warehouse,JathHub_JTH,...,3,Marked_As_RTO,NaN,NaN,NaN,EKLHJAT-1374285,TERMINATED,229078081,OD434135914401094100,itel 5330 | 2.8 inch Big Display | 1900 mAh Ba...
21,FMPP2985566494,ECONOMY,PP,NaN,NORMAL_FORWARD,32019,421302,Undelivered_Not_Attended,Bhiwandi BTS,KhandalaHub_PUNE,...,0,Marked_As_RTO,NaN,NaN,NaN,ESHIUNE-113023,TERMINATED,229222341,OD434139183572892100,HP 15s AMD Ryzen 3 Quad Core 7320U - (8 GB/512...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217065,FMPC4702646305,ECONOMY,COD,NaN,NORMAL_FORWARD,308,282003,Expected,AgraTempMPHub_AGR,ElasticRunUranODH_MUM,...,0.0,Marked_As_RTO,NaN,NaN,NaN,EBOMURN-1754669,REACHED,229121889.0,OD334102256766378100,K' Footlance Women Flats.
217066,FMPC4703025717,ECONOMY,COD,NaN,NORMAL_FORWARD,365,302012,Expected,VKIAMPHub_JAI,MotherHub_HYP,...,0.0,Marked_As_RTO,NaN,NaN,NaN,EJAIBTS-4816230,IN_TRANSIT,230865495.0,OD434104442253823100,BLUE MARTIN Men Checkered Casual Multicolor Sh...
217071,FMPC4702279069,ECONOMY,COD,NaN,NORMAL_FORWARD,209,125001,Undelivered_Order_Rejected_By_Customer,HissarMPHub_HSS,DikomMDH_DIB,...,1.0,Marked_As_RTO,NaN,NaN,NaN,EGAUPLS-3048230,CLOSED,NaN,OD334102339759565100,sunisa Mushroom Head Make up Air Cushion BB Cr...
217072,FMPC4702318656,ECONOMY,COD,NaN,NORMAL_FORWARD,609,562106,Untraceable,JiganiMPHub_BLR,DalhousieMDH_CHM,...,1.0,Marked_As_RTO,Updated by cron,NaN,NaN,ECHMMDA-508416,TERMINATED,229734054.0,OD334103683903972100,HIGHLANDER Men Cargos.


In [59]:
Gunti_Multitracked.to_excel("Gunti_Multitracked.xlsx", index=False)#saturday


In [60]:
CV_Multitracked = Manual[Manual['ShipmentId'].isin(New4['ShipmentId'])].drop_duplicates(subset=['ShipmentId'])#saturday
CV_Multitracked

,ShipmentId,TierType,Type,ReverseShipmentId,ForwardReverseType,TotalPrice,DeliveryPincode,Status,DeliveryHub,CurrentHub,...,NumberOfAttempts,CsNotes,HubNotes,OnHoldByOpsReason,OnHoldByOpsDate,BagId,BagStatus,ConsignmentId,OrderId,ItemDescription
21,FMPP2985566494,ECONOMY,PP,NaN,NORMAL_FORWARD,32019,421302,Undelivered_Not_Attended,Bhiwandi BTS,KhandalaHub_PUNE,...,0,Marked_As_RTO,NaN,NaN,NaN,ESHIUNE-113023,TERMINATED,229222341,OD434139183572892100,HP 15s AMD Ryzen 3 Quad Core 7320U - (8 GB/512...
37,FMPP2991544091,ECONOMY,PP,FMPR0712537067,NORMAL_FORWARD,6345,387570,Undelivered_Order_Rejected_By_Customer,Ahmedabad_01,ElasticRunNarodaODH_NRD,...,2,Marked_As_RTO,NaN,NaN,NaN,XAMDNRD-1805782,TERMINATED,229642922,OD434164334124589100,Infinix Note 50x 5G+ (Enchanted Purple 128 GB).
44,FMPP2983027646,ECONOMY,PP,NaN,NORMAL_FORWARD,765,382220,Expected,GroceryMH_AHM,PrincySanandODH_SND,...,0,Marked_As_RTO,NaN,NaN,NaN,1417-59838,LOST,228673343,OD434109796205481200 OD434109796205481200 OD43...,Ghadi Detergent Cake 250 x 5 Plain Detergent B...
47,FMPP2982406554,ECONOMY,PP,NaN,NORMAL_FORWARD,2679,421302,Expected,BhiwandiMPHub_BHI,MotherHub_PUNE,...,0,Marked_As_RTO,Marked_as_Received_Damaged,NaN,NaN,EMUMSAI4-7133528,IN_TRANSIT,230858317,OD334128627352585100,PUMA BMW Motorsport Drift Cat 5 Ultra Motorspo...
48,FMPP2986940537,ECONOMY,PP,NaN,NORMAL_FORWARD,6875,421101,Undelivered_Request_For_Reschedule,Bhiwandi BTS,BenipattiMDH_MBI,...,1,Marked_As_RTO,NaN,NaN,NaN,NaN,NaN,NaN,OD334146426400540100,Crompton AQUAGOLD DURA 5023 Centrifugal Water ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217009,FMPC4702440600,ECONOMY,COD,NaN,NORMAL_FORWARD,394,781122,Undelivered_No_Response,GroceryMH_GUW,DhunaguriMDH_SPT,...,4.0,Marked_As_RTO,NaN,NaN,NaN,934-GNL-394672,CLOSED,228099834.0,OD434086246820121200 OD434086246820121200 OD43...,Indie Flavors by Flipkart Tasty Crispy Masala ...
217017,FMPC4702236327,ECONOMY,COD,NaN,NORMAL_FORWARD,2712,122503,Undelivered_Not_Attended,sanpka 01,Motherhub_LKO_BTS,...,0.0,Marked_As_RTO,NaN,NaN,NaN,Egur_san-2576071,REACHED,230915093.0,OD334102914636503100,PUMA Puma Fierce runner Running Shoes For Men.
217021,FMPC4702433802,ECONOMY,COD,NaN,NORMAL_FORWARD,2489,741249,Undelivered_Shipment_Damage,Kalash_East_Haringhata_FC_1,DulhinganjMDH_GAY,...,0.0,Marked_As_RTO,Marked_as_Received_Damaged,NaN,NaN,EDUGMDH-566475,TERMINATED,228237823.0,OD434103563605429100 OD434103563605429100 OD43...,PATANJALI Cow's Ghee Danedar Rich Aroma Nat...
217035,FMPC4702147970,ECONOMY,COD,NaN,NORMAL_FORWARD,11558,209859,Undelivered_Order_Rejected_By_Customer,Lucknow FC 02,ShaktiNagarMDH_SNG,...,2.0,Marked_As_RTO,NaN,NaN,NaN,ESNGCNG-671573,TERMINATED,228235497.0,OD434102232551941100,Infinix Note 50x 5G+ (Titanium Grey 128 GB).


In [61]:
CV_Multitracked.to_csv("MSTCV.csv")#saturday
# CV_Multitracked.to_excel("CV_Multitracked.xlsx")


In [62]:
BRSNR_Multitracked = Manual[Manual['ShipmentId'].isin(New5['ShipmentId'])].drop_duplicates(subset=['ShipmentId'])#saturday
BRSNR_Multitracked

,ShipmentId,TierType,Type,ReverseShipmentId,ForwardReverseType,TotalPrice,DeliveryPincode,Status,DeliveryHub,CurrentHub,...,NumberOfAttempts,CsNotes,HubNotes,OnHoldByOpsReason,OnHoldByOpsDate,BagId,BagStatus,ConsignmentId,OrderId,ItemDescription
3,FMPP2989454733,ECONOMY,PP,NaN,NORMAL_FORWARD,242,781124,Expected,Guwahati FC 02,KharupetiyaMDH_MGL,...,0,Marked_As_RTO,NaN,NaN,NaN,EMGLHBV-730650,OPENED,229341831,OD334155107259767100,L'Or?al Paris Casting Creme Gloss Hair Color S...
681,FMPP2991413013,ECONOMY,PP,NaN,NORMAL_FORWARD,216,302026,Undelivered_Not_Attended,Jaipur Sanganer NLFC 01,IdentifyPlusRajsamandODH_RJS,...,0,Marked_As_RTO,NaN,NaN,NaN,EJAIBTS-4820189,CLOSED,NaN,OD434164850167771100,POND's Bright Beauty with vitamin B3 & Niacina...
1206,FMPR0696665810,REGULAR,RVP,NaN,PICKUP_ONLY,276,322201,Untraceable_BRSNR,TTSPLGangapurODH_GAG,TTSPLGangapurODH_GAG,...,1,NaN,Updated by cron,NaN,NaN,EJAIGAP-1781867,OPENED,222156456,OD433779141019175100,NKP Casual Printed Women Maroon Top.
1726,FMPR0681838284,REGULAR,RVP,NaN,PICKUP_ONLY,373,635112,Untraceable_BRSNR,KaveripattinamMDH_KVI,KaveripattinamMDH_KVI,...,1,NaN,Updated by cron,NaN,NaN,EKAVMDH-377428,OPENED,226179548,OD333438277689721100,aadityacopies UNRULED LONGSIZE Regular Noteboo...
1784,FMPR0696908215,REGULAR,RVP,NaN,PICKUP_ONLY,267,322201,Untraceable_BRSNR,TTSPLGangapurODH_GAG,TTSPLGangapurODH_GAG,...,2,NaN,Updated by cron,NaN,NaN,EJAIGAP-1783874,OPENED,223047646,OD333759708983985100,NKP Casual Printed Women Maroon Top.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213725,FMPC4627936541,ECONOMY,COD,NaN,NORMAL_FORWARD,1866,741249,Expected,Kalash_East_Haringhata_FC_1,TTSPLSrinagarODH_SRG,...,0.0,Marked_As_RTO,NaN,NaN,NaN,ELUHLUH-663623743-1775,OPENED,222131280.0,OD433891806744139100,PUMA Zarsun Sneakers For Men.
214011,FMPC4633494947,ECONOMY,COD,NaN,NORMAL_FORWARD,331,302029,Expected,SitapuraMPHub_JAI,ShadowFaxGhatkoparODH_MUM,...,0.0,Marked_As_RTO,NaN,NaN,NaN,EMUMGHA-356427,OPENED,223404685.0,OD333896503596365400,Women Kurta and Pant Set.
214018,FMPC4618956214,ECONOMY,COD,NaN,NORMAL_FORWARD,1766,124103,Expected,Farrukhnagar BTS Warehouse,IdentifyPlusNautanwaODH_NTN,...,0.0,Marked_As_RTO,NaN,NaN,NaN,ELKOBTS-877958-1804,TERMINATED,221657988.0,OD433865831931439100 OD433865831931439100 OD43...,SUNSILK Stunning Black Shine Shampoo Activ-Mix...
215173,FMPC4701824606,ECONOMY,COD,NaN,NORMAL_FORWARD,646.98,302003,Undelivered_Not_Attended,HawamahalMPHub_JAI,ChhatralMDH_CHT,...,0.0,Marked_As_RTO,NaN,NaN,NaN,EAHDABD-3981966,CLOSED,NaN,OD334102790865442100 OD334102790865442100 OD33...,Moncada Women Flats. Moncada Women Flats. Monc...


In [63]:
BRSNR_Multitracked.to_csv("BRSNR_Multitracked.csv")

In [64]:
Map.columns=[' CurrentHub','Vendor Name']
Automa[' CurrentHub']=np.where(Automa[' CurrentHub'].isna()==True,"Dummy",Automa[' CurrentHub'])
Automa[' CurrentHub']=Automa[' CurrentHub'].apply(lambda x:x.upper())
#Automa=Automa.merge(Map,how='left',on=[' CurrentHub'])
Automa = Automa.merge(Map, how='left', on=' CurrentHub')
Flag=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Flag.csv")
Automa[' Status']=np.where(Automa[' Status'].isna()==True,"Dummy",Automa[' Status'])
Automa[' Status']=Automa[' Status'].apply(lambda x:x.upper())
Automa=Automa.merge(Flag,how='left',on=[' Status'])
Automa['Remarks']=np.where(Automa['Vendor Name'].isna()==True,'Clear',
                          np.where(Automa['Flag']==1,'Clear',
                                  np.where((Automa[' BagStatus']=='IN_TRANSIT') | (Automa[' BagStatus']=='OPENED') | (Automa[' BagStatus']=='REACHED'),'Clear','To be considered')))
Automa=Automa.drop(['Flag'],axis=1)

In [65]:
Map.columns=[' CurrentHub','Vendor Name']
Manual[' CurrentHub']=np.where(Manual[' CurrentHub'].isna()==True,"Dummy",Manual[' CurrentHub'])
Manual[' CurrentHub']=Manual[' CurrentHub'].apply(lambda x:x.upper())
#Manual=Manual.merge(Map,how='left',on=[' CurrentHub'])
Manual = Manual.merge(Map, how='left', on=' CurrentHub', suffixes=('', '_MAP'))
Flag=pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Flag.csv")
Flag.columns=[' Status','Flag']
Manual[' Status']=np.where(Manual[' Status'].isna()==True,"Dummy",Manual[' Status'])
Manual[' Status']=Manual[' Status'].apply(lambda x:x.upper())
Manual=Manual.merge(Flag,how='left',on=[' Status'])
Manual['Remarks']=np.where(Manual['Vendor Name'].isna()==True,'Clear',
                          np.where(Manual['Flag']==1,'Clear',
                                  np.where((Manual[' BagStatus']=='IN_TRANSIT') | (Manual[' BagStatus']=='OPENED') | (Manual[' BagStatus']=='REACHED'),'Clear','To be considered')))
Manual=Manual.drop(['Flag'],axis=1)

In [66]:
UT_TIDs['Flag'] = 1
UT_TIDs = UT_TIDs.drop_duplicates()

UT_TIDs.columns = ['ShipmentId', 'Flag'] # Ensure correct column names in UT_TIDs

Automa = Automa.merge(UT_TIDs, how='left', on='ShipmentId', suffixes=('_automa', '_ut')) # Merge Automa with UT_TIDs on 'ShipmentId'

Manual = Manual.merge(UT_TIDs, how='left', on='ShipmentId') # Merge Manual with UT_TIDs on 'ShipmentId' instead of 'Tracking Id'
#Manual['Remarks'] = np.nan


Automa['Remarks'] = np.where((Automa['Flag'] == 1) & (Automa[' Status'] != 'UNTRACEABLE'), 'Clear', Automa['Remarks'])
Manual['Remarks'] = np.where((Manual['Flag'] == 1) & (Manual[' Status'] != 'UNTRACEABLE'), 'Clear', Manual['Remarks']) # Apply remarks condition


Automa = Automa.drop(['Flag'], axis=1)  # Drop 'Flag' column after merging
Manual = Manual.drop(['Flag'], axis=1)



In [67]:
Automa_Map=Automa[['ShipmentId','Remarks']]
Manual_Map=Manual[['ShipmentId','Remarks']]
Manual_Map.columns=['ShipmentId','Remarks2']
RTO_RVP=RTO_RVP.merge(Automa_Map,how='left',on='ShipmentId')
RTO_RVP=RTO_RVP.merge(Manual_Map,how='left',on='ShipmentId')

In [68]:
UT=UT.merge(Automa_Map,how='left',on='ShipmentId')
UT=UT.merge(Manual_Map,how='left',on='ShipmentId')

In [69]:
RTO_RVP['Final_Remarks']=np.where(RTO_RVP['Remarks'].isna()==True,RTO_RVP['Remarks2'],RTO_RVP['Remarks'])
RTO_RVP=RTO_RVP.drop(['Remarks','Remarks2'],axis=1)
UT['Final_Remarks']=np.where(UT['Remarks'].isna()==True,UT['Remarks2'],UT['Remarks'])
UT=UT.drop(['Remarks','Remarks2'],axis=1)

In [70]:
RTO_RVP_Final=RTO_RVP[RTO_RVP['Final_Remarks']!='Clear']
RTO_RVP_Final=RTO_RVP_Final[(RTO_RVP_Final['Month']>202501)]
UT_Final=UT[UT['Final_Remarks']!='Clear']
UT_Final=UT_Final[UT_Final['CurrentShipmentStatus']=='Untraceable']
UT_Final=UT_Final[(UT_Final['Month']>202501)]
Automa=Automa[Automa['Remarks']!='Clear']
Manual=Manual[Manual['Remarks']!='Clear']

In [71]:
RTO_RVP_Final['LastUpdatDateTime']=np.where(RTO_RVP_Final['TypeOfShipment']=="RTO",RTO_RVP_Final['RTOCreatedDateTime'],RTO_RVP_Final['LastUpdatDateTime'])
RTO_RVP_Final=RTO_RVP_Final[['ShipmentId','HubName','CurrentStatus','LastUpdatDateTime','ShipmentValue','TypeOfShipment','Vendor Name','Month','Ageing']]
RTO_RVP_Final.columns=['ShipmentId','HubName','CurrentStatus','LastUpdatDateTime','ShipmentValue','TypeOfShipment','Vendor Name','Month','Ageing']
UT_Final=UT_Final[['ShipmentId','HubName','CurrentShipmentStatus','CurrentStatusUpdateDateTime','ShipmentAmount','CurrentShipmentStatus','Vendor Name','Month','Ageing']]
UT_Final.columns=['ShipmentId','HubName','CurrentStatus','LastUpdatDateTime','ShipmentValue','TypeOfShipment','Vendor Name','Month','Ageing']

In [72]:
print("Manual Columns:", Manual.columns)

Manual Columns: Index(['ShipmentId', ' TierType', ' Type', ' ReverseShipmentId',
       ' ForwardReverseType', ' TotalPrice', ' DeliveryPincode', ' Status',
       ' DeliveryHub', ' CurrentHub', ' CustomerPromiseDate',
       ' LogisticsPromiseDate', ' LatestUpdateTime', ' FirstReceiveTime',
       ' FirstReceiveHub', ' LastReceiveTime', ' FirstAssignedHub',
       ' NumberOfAttempts', ' CsNotes', ' HubNotes', ' OnHoldByOpsReason',
       ' OnHoldByOpsDate', ' BagId', ' BagStatus', ' ConsignmentId',
       ' OrderId', ' ItemDescription', 'Vendor Name', 'Remarks'],
      dtype='object')


In [73]:
# Attempt to parse with `errors='coerce'`
Manual[' LatestUpdateTime'] = pd.to_datetime(Manual[' LatestUpdateTime'], errors='coerce')

# Check for unparseable entries
invalid_entries = Manual[Manual[' LatestUpdateTime'].isna()]
if not invalid_entries.empty:
    print("Unparseable datetime entries found:", invalid_entries)

# Proceed if column is clean
Manual['TimeDelta'] = dt.now() - Manual[' LatestUpdateTime']
Manual['Ageing'] = Manual['TimeDelta'].apply(lambda x: float(x.days) + float(x.seconds) / (60 * 60 * 24))
Manual['Month'] = Manual[' LatestUpdateTime'].dt.month.astype(str)
Manual['Year'] = Manual[' LatestUpdateTime'].dt.year.astype(str)


In [74]:
Automa['Month']=pd.DatetimeIndex(Automa[' LatestUpdateTime']).month
Automa['Year']=pd.DatetimeIndex(Automa[' LatestUpdateTime']).year
Automa['TimeDelta']=dt.now()-pd.to_datetime(Automa[' LatestUpdateTime'])
Automa['Ageing']=Automa['TimeDelta'].apply(lambda x: float(x.days)+float(x.seconds)/(60*60*24))
Automa['Month']=Automa['Month'].astype(str)
Automa['Length']=Automa['Month'].apply(lambda x:len(x))
Automa['Month']=np.where(Automa['Length']==2,Automa['Year'].astype(str)+Automa['Month'].astype(str),
                         Automa['Year'].astype(str)+"0"+Automa['Month'].astype(str))
Manual['Month']=pd.DatetimeIndex(Manual[' LatestUpdateTime']).month
Manual['Year']=pd.DatetimeIndex(Manual[' LatestUpdateTime']).year
Manual['TimeDelta']=dt.now()-pd.to_datetime(Manual[' LatestUpdateTime'])
Manual['Ageing']=Manual['TimeDelta'].apply(lambda x: float(x.days)+float(x.seconds)/(60*60*24))
Manual['Month']=Manual['Month'].astype(str)
Manual['Length']=Manual['Month'].apply(lambda x:len(x))
Manual['Month']=np.where(Manual['Length']==2,Manual['Year'].astype(str)+Manual['Month'].astype(str),
                         Manual['Year'].astype(str)+"0"+Manual['Month'].astype(str))

In [75]:
# Remove leading and trailing spaces from column names
Automa.columns = Automa.columns.str.strip()
Manual.columns = Manual.columns.str.strip()

# Select relevant columns
Automa = Automa[['ShipmentId', 'CurrentHub', 'Status', 'LatestUpdateTime', 
                 'TotalPrice', 'Vendor Name', 'Month', 'Ageing', 'DeliveryHub']]

Manual = Manual[['ShipmentId', 'CurrentHub', 'Status', 'LatestUpdateTime', 
                 'TotalPrice', 'Vendor Name', 'Month', 'Ageing', 'DeliveryHub']]

# Concatenate both DataFrames
Tracking = pd.concat([Automa, Manual], ignore_index=True)


C:\Users\prajwal.d1\AppData\Local\Temp\ipykernel_23300\768821000.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Tracking = pd.concat([Automa, Manual], ignore_index=True)


In [76]:
UT_Map=UT[['ShipmentId']]
UT_Map=UT_Map.drop_duplicates()
UT_Map['Flag']=1
Tracking=Tracking.merge(UT_Map,how='left',on='ShipmentId')
Tracking1=sqldf("select * from Tracking where DeliveryHub like '%Grocery%'")
Tracking2=sqldf("select * from Tracking where DeliveryHub not like '%Grocery%'")
Tracking1['Type']='GTNL'
Tracking2['Type']=np.where(Tracking2['Flag']==1,'Untraceable',
                         np.where(Tracking2['ShipmentId'].str.slice(3,4)=="R","RVP","RTO"))
Tracking=pd.concat([Tracking1,Tracking2])

In [77]:


# Now select the required columns
Tracking = Tracking[['ShipmentId', 'CurrentHub', 'Status', 'LatestUpdateTime', 
                     'TotalPrice', 'Type', 'Vendor Name', 'Month', 'Ageing']]

# Rename the columns
Tracking.columns = ['ShipmentId', 'HubName', 'CurrentStatus', 'LastUpdatDateTime', 
                    'ShipmentValue', 'TypeOfShipment', 'Vendor Name', 'Month', 'Ageing']

# Convert 'Month' column to numeric, handling errors
Tracking['Month'] = pd.to_numeric(Tracking['Month'], errors='coerce')

# Drop rows with invalid or missing 'Month' values
Tracking = Tracking.dropna(subset=['Month'])

# Convert 'Month' to integers
Tracking['Month'] = Tracking['Month'].astype(int)

# Filter rows where 'Month' > 202411
Tracking = Tracking[Tracking['Month'] > 202501]


In [78]:
Final=pd.concat([RTO_RVP_Final,UT_Final])
Final=pd.concat([Final,Tracking])
Final=Final.drop_duplicates(subset=['ShipmentId'])
Final['Slab']=np.where(Final['Ageing']<=2,"0 to 2",
                      np.where(Final['Ageing']<=5,"2 to 5",">5"))
#New1_TIDs=pd.concat([New1_TIDs,Prexo_TIDs])
New1_TIDs['Model']='FA'
Final=Final.merge(New1_TIDs,how='left',on='ShipmentId')
Final['Model']=np.where(Final['Model'].isna()==True,"Others",Final['Model'])

In [79]:
Debit = pd.read_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Debit_data.csv", encoding="ISO-8859-1")
Debit=Debit.drop_duplicates()
Debit['ShipmentId']=Debit['ShipmentId'].apply(lambda x:x.upper())
Final=Final.merge(Debit,how='left',on='ShipmentId')
Final=Final[Final['Flag']!=1]
Final=Final.drop(['Flag'],axis=1)
#Final.to_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Cron_Data.csv") ##end

In [80]:
New3 = New3[New3["Asset"]=="ODH"]
df_g = New3[['ShipmentId',' TotalPrice','source']]
df_g = pd.merge(df_g, Manual1[[" Status", " CurrentHub", "ShipmentId", ' LatestUpdateTime']], on = 'ShipmentId', how='left')
df_g.rename(columns = {' TotalPrice':'ShipmentValue','source':'Model'," Status":'CurrentStatus', " CurrentHub":'HubName', ' LatestUpdateTime':'LastUpdatDateTime'},inplace=True)
df_g["HubName"] = df_g["HubName"].str.upper()
df_g["CurrentStatus"] = df_g["CurrentStatus"].str.upper()

In [81]:
Map.rename(columns={' CurrentHub':'HubName'}, inplace=True)
df_g = pd.merge(df_g, Map, on = 'HubName', how='left')
df_g = df_g.dropna(subset={'Vendor Name'}, how = 'all')

df_g['LastUpdatDateTime'] = pd.to_datetime(df_g['LastUpdatDateTime'], errors='coerce')
df_g['Month']= df_g['LastUpdatDateTime'].dt.year*100 + df_g['LastUpdatDateTime'].dt.month

df_g['LastUpdatDateTime'] = pd.to_datetime(df_g['LastUpdatDateTime'])
df_g['Ageing']= pd.to_datetime(dt.now()) - df_g['LastUpdatDateTime']
df_g['Ageing'] = df_g['Ageing'] / pd.Timedelta(days=1)

In [82]:
def assign_slab(days):
    if days > 5:
        return '>5'
    elif days > 2:
        return '2 to 5'
    else:
        return '0 to 2'

In [83]:
df_g['Slab'] = df_g['Ageing'].apply(assign_slab)

df_g['TypeOfShipment']=np.where(df_g['ShipmentId'].str.slice(3,4)=="R","RVP","RTO")

Flag.rename(columns={' Status':"CurrentStatus"},inplace=True)
df_g = df_g.merge(Flag[["CurrentStatus", "Flag"]], on="CurrentStatus", how="left")
df_g = df_g[df_g["Flag"]!=1.0]
df_g = df_g[['ShipmentId', 'ShipmentValue', 'Model', 'CurrentStatus', 'HubName','LastUpdatDateTime', 'Vendor Name', 'Month', 'Ageing', 'Slab', 'TypeOfShipment']]

Final = pd.concat([Final, df_g], ignore_index = True, sort = False)
Final = Final.drop_duplicates("ShipmentId")

Final.to_csv("C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Cron_Data.csv") #TheEnd